## Author: Tao Shan

## Updates

* Update v0_5_BERT
  * this version only has BERT but no CNN

* Update v0_5

  * delete batch normalization and layers after contact

* Update v0_4

  * Increase number of samples

* Update v0_3

  * base on v0_2, update preprocessing utilize GPU, and add early stopping, improve final result
* Update v0_2

  * Combine TMDB Dataset, including description, movie description and posters
  * Combine Deep Learning models, include BERT and CNN (VGG16)

* Update v0_1

  * deep learning
* Initial v0_0
  
  * Initial data to use, including score, movie and movie's information
  * Initial preprocessing
  * Initial Models

## Description: 
Combined Model Analysis

Data overview for movie len dataset: 

https://colab.research.google.com/drive/16HjJxswtEQ-CGm-S7O6LYerYHg7KPMyd?usp=sharing

Data generation from TMDB API:

https://colab.research.google.com/drive/1RlT_bupC2LzHqXeiHXhPRSu7fCJw_5TZ?usp=sharing

# Clean and Split dataset


*   df_metadata: movie information
*   df_text: movie text data
*   df_figures: movie posters


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
DIR_PATH = "/content/drive/MyDrive/cs680 final project/MovieLen_25M_data"

In [3]:
import os
import sys
import pandas as pd
import numpy as np

In [4]:
# Rating Information
df_ratings = pd.read_csv(
    os.path.join(DIR_PATH, f"ratings.csv")
)
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [5]:
df_ratings.shape

(25000095, 4)

In [6]:
# choose a small sample
# sample = 10000
# df_ratings = df_ratings.iloc[:sample,:]

In [7]:
# use first 100 user's data, because of limit in RAM (the final data potentially at least 10 GB)
unique_user_ids = df_ratings['userId'].unique()[:100]
df_ratings = df_ratings[df_ratings['userId'].isin(unique_user_ids)]
df_ratings.shape

(13296, 4)

In [8]:
# link between TMDB and Movie Len
df_links = pd.read_csv(
    os.path.join(DIR_PATH, f"links.csv")
)
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [9]:
# TMDB dataset
DIR_PATH_TMDB =  "/content/drive/MyDrive/cs680 final project/TMDB_data"

df_metadata = pd.read_csv(
    os.path.join(DIR_PATH_TMDB, f"movie_data.csv")
)
df_metadata.reset_index(inplace = True, drop = True)
df_metadata.head()

<ipython-input-9-3c56f963bb82>:4: DtypeWarning: Columns (0,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metadata = pd.read_csv(


,id,title,original_title,genres,overview,tagline,release_date,runtime,popularity,vote_count,vote_average,poster_path,poster_url,original_language,status,production_companies,production_countries,spoken_languages
0,680,Pulp Fiction,Pulp Fiction,"['Thriller', 'Crime']","A burger-loving hit man, his philosophical par...",Just because you are a character doesn't mean ...,1994-09-10,154.0,77.661,24936.0,8.49,/d5iIlFn5s0ImszYzBPb8JPIfbXD.jpg,https://image.tmdb.org/t/p/w500/d5iIlFn5s0Imsz...,en,Released,"['Miramax', 'A Band Apart', 'Jersey Films']",['United States of America'],"['English', 'Spanish', 'French']"
1,110,Three Colors: Red,Trois couleurs : Rouge,"['Drama', 'Mystery', 'Romance']","Valentine, a student model in Geneva, struggle...",NaN,1994-05-12,100.0,17.682,1149.0,7.956,/JHmsBiX1tjCKqAul1lzC20WcAW.jpg,https://image.tmdb.org/t/p/w500/JHmsBiX1tjCKqA...,fr,Released,"['Zespół Filmowy TOR', 'Le Studio Canal+', 'Fr...","['France', 'Poland', 'Switzerland']",['French']
2,108,Three Colors: Blue,Trois couleurs : Bleu,['Drama'],Julie is haunted by her grief after living thr...,NaN,1993-08-01,98.0,15.606,1412.0,7.689,/33wsWxzsNstI8N7dvuwzFmj1qBd.jpg,https://image.tmdb.org/t/p/w500/33wsWxzsNstI8N...,fr,Released,"['France 3 Cinéma', 'CED Productions', 'Miramax']","['France', 'Poland', 'Switzerland']","['French', 'Polish']"
3,11902,Underground,Подземље,"['Comedy', 'Drama', 'War']",A group of Serbian socialists prepares for the...,ONCE UPON A TIME THERE WAS A COUNTRY,1995-04-11,170.0,10.753,554.0,7.741,/h8N6y13t4VusrDdH5PzTkwvBvgN.jpg,https://image.tmdb.org/t/p/w500/h8N6y13t4VusrD...,sr,Released,"['Komuna', 'Barrandov Studio']","['Bulgaria', 'Czech Republic', 'France', 'Germ...","['French', 'German', 'Serbian']"
4,872,Singin' in the Rain,Singin' in the Rain,"['Comedy', 'Music', 'Romance', 'Drama']","In 1927 Hollywood, a silent film production co...",What a Glorious Feeling!,1952-04-09,103.0,22.318,2710.0,8.165,/671EPwBsHGHBk0cdOeZqmOK0XB3.jpg,https://image.tmdb.org/t/p/w500/671EPwBsHGHBk0...,en,Released,['Metro-Goldwyn-Mayer'],['United States of America'],['English']


In [10]:
df_metadata.shape

(66657, 18)

In [11]:
df_metadata['id'].nunique()

23876

In [12]:
df_metadata[df_metadata['id'].duplicated(keep=False)]['id'].value_counts()

40432     26157
122662    16626
Name: id, dtype: int64

In [13]:
df_metadata[df_metadata['id'] == 122662].head()

,id,title,original_title,genres,overview,tagline,release_date,runtime,popularity,vote_count,vote_average,poster_path,poster_url,original_language,status,production_companies,production_countries,spoken_languages
16142,122662,Mardock Scramble: The Third Exhaust,マルドゥック・スクランブル 排気,"['Animation', 'Science Fiction']",Third film of the Mardock Scramble series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16143,122662,Mardock Scramble: The Third Exhaust,マルドゥック・スクランブル 排気,"['Animation', 'Science Fiction']",Third film of the Mardock Scramble series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16144,122662,Mardock Scramble: The Third Exhaust,マルドゥック・スクランブル 排気,"['Animation', 'Science Fiction']",Third film of the Mardock Scramble series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16145,122662,Mardock Scramble: The Third Exhaust,マルドゥック・スクランブル 排気,"['Animation', 'Science Fiction']",Third film of the Mardock Scramble series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16146,122662,Mardock Scramble: The Third Exhaust,マルドゥック・スクランブル 排気,"['Animation', 'Science Fiction']",Third film of the Mardock Scramble series.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# merge the text words
df_metadata['overview'] = df_metadata['title'].fillna('')+' '+df_metadata['overview'].fillna('')+' '+df_metadata['tagline'].fillna('')
df_metadata.drop(['title','original_title','tagline'], axis = 1, inplace = True)
# from above, missing value is because of error data

In [15]:
df_metadata.dropna(inplace = True)

In [16]:
df_metadata.shape

(23872, 15)

In [17]:
df_metadata.reset_index(inplace = True, drop = True)

In [18]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23872 entries, 0 to 23871
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    23872 non-null  object
 1   genres                23872 non-null  object
 2   overview              23872 non-null  object
 3   release_date          23872 non-null  object
 4   runtime               23872 non-null  object
 5   popularity            23872 non-null  object
 6   vote_count            23872 non-null  object
 7   vote_average          23872 non-null  object
 8   poster_path           23872 non-null  object
 9   poster_url            23872 non-null  object
 10  original_language     23872 non-null  object
 11  status                23872 non-null  object
 12  production_companies  23872 non-null  object
 13  production_countries  23872 non-null  object
 14  spoken_languages      23872 non-null  object
dtypes: object(15)
memory usage: 2.7+ MB


In [19]:
df_metadata.head()

,id,genres,overview,release_date,runtime,popularity,vote_count,vote_average,poster_path,poster_url,original_language,status,production_companies,production_countries,spoken_languages
0,680,"['Thriller', 'Crime']","Pulp Fiction A burger-loving hit man, his phil...",1994-09-10,154.0,77.661,24936.0,8.49,/d5iIlFn5s0ImszYzBPb8JPIfbXD.jpg,https://image.tmdb.org/t/p/w500/d5iIlFn5s0Imsz...,en,Released,"['Miramax', 'A Band Apart', 'Jersey Films']",['United States of America'],"['English', 'Spanish', 'French']"
1,110,"['Drama', 'Mystery', 'Romance']","Three Colors: Red Valentine, a student model i...",1994-05-12,100.0,17.682,1149.0,7.956,/JHmsBiX1tjCKqAul1lzC20WcAW.jpg,https://image.tmdb.org/t/p/w500/JHmsBiX1tjCKqA...,fr,Released,"['Zespół Filmowy TOR', 'Le Studio Canal+', 'Fr...","['France', 'Poland', 'Switzerland']",['French']
2,108,['Drama'],Three Colors: Blue Julie is haunted by her gri...,1993-08-01,98.0,15.606,1412.0,7.689,/33wsWxzsNstI8N7dvuwzFmj1qBd.jpg,https://image.tmdb.org/t/p/w500/33wsWxzsNstI8N...,fr,Released,"['France 3 Cinéma', 'CED Productions', 'Miramax']","['France', 'Poland', 'Switzerland']","['French', 'Polish']"
3,11902,"['Comedy', 'Drama', 'War']",Underground A group of Serbian socialists prep...,1995-04-11,170.0,10.753,554.0,7.741,/h8N6y13t4VusrDdH5PzTkwvBvgN.jpg,https://image.tmdb.org/t/p/w500/h8N6y13t4VusrD...,sr,Released,"['Komuna', 'Barrandov Studio']","['Bulgaria', 'Czech Republic', 'France', 'Germ...","['French', 'German', 'Serbian']"
4,872,"['Comedy', 'Music', 'Romance', 'Drama']","Singin' in the Rain In 1927 Hollywood, a silen...",1952-04-09,103.0,22.318,2710.0,8.165,/671EPwBsHGHBk0cdOeZqmOK0XB3.jpg,https://image.tmdb.org/t/p/w500/671EPwBsHGHBk0...,en,Released,['Metro-Goldwyn-Mayer'],['United States of America'],['English']


In [20]:
from datetime import datetime
# correct data type
df_metadata['release_date'] = pd.to_datetime(df_metadata['release_date'])
for col in ['runtime','popularity','vote_count','vote_average']:
    df_metadata[col] = df_metadata[col].astype(float)

In [21]:
def frequency_encoding(column):
    # Count the occurrences of each label in the column
    label_counts = {}
    for row in column:
        for label in row:
            if label in label_counts:
                label_counts[label] += 1
            else:
                label_counts[label] = 1
    # Create a frequency-encoded column
    print(label_counts)
    encoded_column = column.apply(lambda row: sum([label_counts[label] for label in row]))
    return encoded_column

In [22]:
from tqdm.auto import tqdm
import ast
total_rows = df_metadata.shape[0]
threshold = 0.05
for col in tqdm(['genres','production_companies','production_countries','spoken_languages']):
    df_metadata[col] = df_metadata[col].apply(ast.literal_eval)
    df_metadata[col] = frequency_encoding(df_metadata[col])
df_metadata.head()

  0%|          | 0/4 [00:00<?, ?it/s]

{'Thriller': 4766, 'Crime': 2996, 'Drama': 11512, 'Mystery': 1747, 'Romance': 4536, 'Comedy': 8311, 'War': 870, 'Music': 1047, 'Science Fiction': 2000, 'Fantasy': 1780, 'Action': 3879, 'History': 1050, 'Adventure': 2457, 'Family': 1971, 'Documentary': 1523, 'Animation': 1307, 'Horror': 2951, 'Western': 435, 'TV Movie': 599, '': 62}
{'Miramax': 245, 'A Band Apart': 11, 'Jersey Films': 24, 'Zespół Filmowy TOR': 2, 'Le Studio Canal+': 9, 'France 3 Cinéma': 108, 'TSR': 15, 'CED Productions': 4, 'Komuna': 2, 'Barrandov Studio': 5, 'Metro-Goldwyn-Mayer': 637, 'Great American Films Limited Partnership': 3, 'Vestron Pictures': 16, 'Victoires Productions': 3, 'Constellation': 3, 'Hachette Première': 11, 'Sofinergie Films': 2, 'Sofinergie 2': 3, 'Investimage 2': 2, 'Fondation GAN pour le Cinéma': 2, 'Union Générale Cinématographique (UGC)': 18, 'Nippon Herald Films': 6, 'Greenwich Film Production': 7, 'Herald Ace': 2, 'TOHO': 162, 'SF Studios': 65, 'Horizon Pictures': 10, 'Columbia Pictures': 70

,id,genres,overview,release_date,runtime,popularity,vote_count,vote_average,poster_path,poster_url,original_language,status,production_companies,production_countries,spoken_languages
0,680,7762,"Pulp Fiction A burger-loving hit man, his phil...",1994-09-10,154.0,77.661,24936.0,8.490,/d5iIlFn5s0ImszYzBPb8JPIfbXD.jpg,https://image.tmdb.org/t/p/w500/d5iIlFn5s0Imsz...,en,Released,280,14370,21999
1,110,17795,"Three Colors: Red Valentine, a student model i...",1994-05-12,100.0,17.682,1149.0,7.956,/JHmsBiX1tjCKqAul1lzC20WcAW.jpg,https://image.tmdb.org/t/p/w500/JHmsBiX1tjCKqA...,fr,Released,134,2702,2521
2,108,11512,Three Colors: Blue Julie is haunted by her gri...,1993-08-01,98.0,15.606,1412.0,7.689,/33wsWxzsNstI8N7dvuwzFmj1qBd.jpg,https://image.tmdb.org/t/p/w500/33wsWxzsNstI8N...,fr,Released,357,2702,2742
3,11902,20693,Underground A group of Serbian socialists prep...,1995-04-11,170.0,10.753,554.0,7.741,/h8N6y13t4VusrDdH5PzTkwvBvgN.jpg,https://image.tmdb.org/t/p/w500/h8N6y13t4VusrD...,sr,Released,7,4047,4002
4,872,25406,"Singin' in the Rain In 1927 Hollywood, a silen...",1952-04-09,103.0,22.318,2710.0,8.165,/671EPwBsHGHBk0cdOeZqmOK0XB3.jpg,https://image.tmdb.org/t/p/w500/671EPwBsHGHBk0...,en,Released,637,14370,17890


In [23]:
df_text = df_metadata[['id','overview']]
df_metadata.drop(['overview','poster_path','poster_url'], axis = 1, inplace = True)
df_metadata.head()

,id,genres,release_date,runtime,popularity,vote_count,vote_average,original_language,status,production_companies,production_countries,spoken_languages
0,680,7762,1994-09-10,154.0,77.661,24936.0,8.490,en,Released,280,14370,21999
1,110,17795,1994-05-12,100.0,17.682,1149.0,7.956,fr,Released,134,2702,2521
2,108,11512,1993-08-01,98.0,15.606,1412.0,7.689,fr,Released,357,2702,2742
3,11902,20693,1995-04-11,170.0,10.753,554.0,7.741,sr,Released,7,4047,4002
4,872,25406,1952-04-09,103.0,22.318,2710.0,8.165,en,Released,637,14370,17890


In [24]:
df_metadata['original_language'].nunique()

71

In [25]:
df_metadata['original_language'].value_counts()[0:5]

en    17287
fr     1253
ja      727
ru      639
de      453
Name: original_language, dtype: int64

In [26]:
df_metadata['is_en'] = (df_metadata['original_language'] == "en") * 1

In [27]:
df_metadata['status'].nunique()

1

In [28]:
df_metadata.drop('status', axis = 1, inplace = True)

In [29]:
def frequency_encoding(column):
    # Count the occurrences of each label in the column
    label_counts = {}
    for label in column:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1
    # Create a frequency-encoded column
    print(label_counts)
    encoded_column = column.apply(lambda row: label_counts[row])
    return encoded_column

In [30]:
df_metadata['original_language'] = frequency_encoding(df_metadata['original_language'])

{'en': 17287, 'fr': 1253, 'sr': 11, 'ja': 727, 'sv': 159, 'de': 453, 'it': 424, 'es': 418, 'pl': 98, 'da': 114, 'cn': 183, 'pt': 99, 'is': 18, 'ko': 283, 'zh': 226, 'mk': 5, 'ru': 639, 'id': 8, 'hi': 414, 'fi': 212, 'he': 41, 'ro': 21, 'bs': 2, 'nl': 76, 'bn': 9, 'cs': 87, 'sh': 10, 'th': 39, 'no': 62, 'tr': 69, 'ar': 23, 'vi': 3, 'fa': 66, 'ps': 1, 'ku': 3, 'hu': 68, 'el': 76, 'af': 2, 'ta': 56, 'nb': 3, 'iu': 1, 'et': 29, 'xx': 12, 'ka': 9, 'uk': 10, 'bo': 5, 'se': 1, 'eu': 2, 'mn': 1, 'mi': 1, 'sk': 1, 'bm': 1, 'hy': 1, 'wo': 2, 'jv': 1, 'tl': 11, 'cy': 1, 'lo': 1, 'te': 7, 'mr': 1, 'ml': 2, 'hr': 3, 'kk': 1, 'dz': 1, 'bg': 2, 'ca': 3, 'lt': 2, 'lv': 5, 'sl': 5, 'rw': 1, 'sq': 1}


In [31]:
df_metadata.head()

,id,genres,release_date,runtime,popularity,vote_count,vote_average,original_language,production_companies,production_countries,spoken_languages,is_en
0,680,7762,1994-09-10,154.0,77.661,24936.0,8.490,17287,280,14370,21999,1
1,110,17795,1994-05-12,100.0,17.682,1149.0,7.956,1253,134,2702,2521,0
2,108,11512,1993-08-01,98.0,15.606,1412.0,7.689,1253,357,2702,2742,0
3,11902,20693,1995-04-11,170.0,10.753,554.0,7.741,11,7,4047,4002,0
4,872,25406,1952-04-09,103.0,22.318,2710.0,8.165,17287,637,14370,17890,1


In [32]:
df_metadata['year'] = df_metadata['release_date'].dt.year
df_metadata['month'] = df_metadata['release_date'].dt.month
df_metadata['day'] = df_metadata['release_date'].dt.day
df_metadata['weekday'] = df_metadata['release_date'].dt.weekday

In [33]:
df_metadata.drop(['release_date'], axis = 1, inplace = True)

movie information

In [34]:
df_metadata.head()

,id,genres,runtime,popularity,vote_count,vote_average,original_language,production_companies,production_countries,spoken_languages,is_en,year,month,day,weekday
0,680,7762,154.0,77.661,24936.0,8.490,17287,280,14370,21999,1,1994,9,10,5
1,110,17795,100.0,17.682,1149.0,7.956,1253,134,2702,2521,0,1994,5,12,3
2,108,11512,98.0,15.606,1412.0,7.689,1253,357,2702,2742,0,1993,8,1,6
3,11902,20693,170.0,10.753,554.0,7.741,11,7,4047,4002,0,1995,4,11,1
4,872,25406,103.0,22.318,2710.0,8.165,17287,637,14370,17890,1,1952,4,9,2


In [35]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23872 entries, 0 to 23871
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    23872 non-null  object 
 1   genres                23872 non-null  int64  
 2   runtime               23872 non-null  float64
 3   popularity            23872 non-null  float64
 4   vote_count            23872 non-null  float64
 5   vote_average          23872 non-null  float64
 6   original_language     23872 non-null  int64  
 7   production_companies  23872 non-null  int64  
 8   production_countries  23872 non-null  int64  
 9   spoken_languages      23872 non-null  int64  
 10  is_en                 23872 non-null  int64  
 11  year                  23872 non-null  int64  
 12  month                 23872 non-null  int64  
 13  day                   23872 non-null  int64  
 14  weekday               23872 non-null  int64  
dtypes: float64(4), int6

text words

In [36]:
df_text.head()

,id,overview
0,680,"Pulp Fiction A burger-loving hit man, his phil..."
1,110,"Three Colors: Red Valentine, a student model i..."
2,108,Three Colors: Blue Julie is haunted by her gri...
3,11902,Underground A group of Serbian socialists prep...
4,872,"Singin' in the Rain In 1927 Hollywood, a silen..."


# Preprocess data

## Combine movie data and rating data

In [37]:
df = df_ratings.merge(df_links, on='movieId', how='left')
df.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,1,296,5.0,1147880044,110912,680.0
1,1,306,3.5,1147868817,111495,110.0
2,1,307,5.0,1147868828,108394,108.0
3,1,665,5.0,1147878820,114787,11902.0
4,1,899,3.5,1147868510,45152,872.0


In [38]:
df.isnull().sum().sort_values(ascending = False)/df.shape[0]

userId       0.0
movieId      0.0
rating       0.0
timestamp    0.0
imdbId       0.0
tmdbId       0.0
dtype: float64

In [39]:
df.dropna(inplace = True)

In [40]:
df['tmdbId'] = df['tmdbId'].astype('int').astype('object')
df.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,1,296,5.0,1147880044,110912,680
1,1,306,3.5,1147868817,111495,110
2,1,307,5.0,1147868828,108394,108
3,1,665,5.0,1147878820,114787,11902
4,1,899,3.5,1147868510,45152,872


In [41]:
df_metadata = df.merge(df_metadata, left_on='tmdbId', right_on = 'id', how='left')
df_metadata.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId,id,genres,runtime,popularity,...,vote_average,original_language,production_companies,production_countries,spoken_languages,is_en,year,month,day,weekday
0,1,296,5.0,1147880044,110912,680,680,7762.0,154.0,77.661,...,8.490,17287.0,280.0,14370.0,21999.0,1.0,1994.0,9.0,10.0,5.0
1,1,306,3.5,1147868817,111495,110,110,17795.0,100.0,17.682,...,7.956,1253.0,134.0,2702.0,2521.0,0.0,1994.0,5.0,12.0,3.0
2,1,307,5.0,1147868828,108394,108,108,11512.0,98.0,15.606,...,7.689,1253.0,357.0,2702.0,2742.0,0.0,1993.0,8.0,1.0,6.0
3,1,665,5.0,1147878820,114787,11902,11902,20693.0,170.0,10.753,...,7.741,11.0,7.0,4047.0,4002.0,0.0,1995.0,4.0,11.0,1.0
4,1,899,3.5,1147868510,45152,872,872,25406.0,103.0,22.318,...,8.165,17287.0,637.0,14370.0,17890.0,1.0,1952.0,4.0,9.0,2.0


## Combine movie data and text data

In [42]:
df_text = df.merge(df_text, left_on='tmdbId', right_on = 'id', how='left')
df_text.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId,id,overview
0,1,296,5.0,1147880044,110912,680,680,"Pulp Fiction A burger-loving hit man, his phil..."
1,1,306,3.5,1147868817,111495,110,110,"Three Colors: Red Valentine, a student model i..."
2,1,307,5.0,1147868828,108394,108,108,Three Colors: Blue Julie is haunted by her gri...
3,1,665,5.0,1147878820,114787,11902,11902,Underground A group of Serbian socialists prep...
4,1,899,3.5,1147868510,45152,872,872,"Singin' in the Rain In 1927 Hollywood, a silen..."


In [43]:
df_metadata.isnull().sum().sort_values(ascending = False)/df.shape[0]

vote_count              0.001805
vote_average            0.001805
day                     0.001805
month                   0.001805
year                    0.001805
is_en                   0.001805
spoken_languages        0.001805
production_countries    0.001805
production_companies    0.001805
original_language       0.001805
weekday                 0.001805
popularity              0.001805
runtime                 0.001805
genres                  0.001805
id                      0.001805
movieId                 0.000000
tmdbId                  0.000000
imdbId                  0.000000
timestamp               0.000000
rating                  0.000000
userId                  0.000000
dtype: float64

In [44]:
df_text.isnull().sum().sort_values(ascending = False)/df_text.shape[0]

id           0.001805
overview     0.001805
userId       0.000000
movieId      0.000000
rating       0.000000
timestamp    0.000000
imdbId       0.000000
tmdbId       0.000000
dtype: float64

In [45]:
df.dropna(inplace = True)
df_text.dropna(inplace = True)

In [46]:
df_metadata.dropna(inplace = True)

In [47]:
df_text.reset_index(inplace = True, drop = True)
df_text.shape

(13272, 8)

In [48]:
df_metadata.reset_index(inplace = True, drop = True)
df_metadata.shape

(13272, 21)

## Preprocess text data

In [49]:
df_text.shape

(13272, 8)

In [50]:
df_text.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId,id,overview
0,1,296,5.0,1147880044,110912,680,680,"Pulp Fiction A burger-loving hit man, his phil..."
1,1,306,3.5,1147868817,111495,110,110,"Three Colors: Red Valentine, a student model i..."
2,1,307,5.0,1147868828,108394,108,108,Three Colors: Blue Julie is haunted by her gri...
3,1,665,5.0,1147878820,114787,11902,11902,Underground A group of Serbian socialists prep...
4,1,899,3.5,1147868510,45152,872,872,"Singin' in the Rain In 1927 Hollywood, a silen..."


In [51]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [52]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from tqdm.auto import tqdm
tqdm.pandas()
# lowercase
def lower(df, cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: x.lower())
    return df

# number removing
def num_remove(df, cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: re.sub(r'\d+', '', x))
    return df

# punctuation removal
def punc_remove(df,cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: re.sub('['+string.punctuation+']', ' ', x))
    return df

# white spaces removal
def space_remove(df,cols):
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: x.strip()) # remove front and end space
        df[col] = df[col].str.replace('\s+', ' ', regex=True) # remove double space
    return df

def stop(string):
    stops = set(stopwords.words('english'))
    tokens = word_tokenize(string)
    result = [i for i in tqdm(tokens) if not i in stops]
    return result
    
def stop_remove(df,cols):
    stops = set(stopwords.words('english'))
    for col in tqdm(cols):
        df[col] = df[col].progress_apply(lambda x: ' '.join([word for word in x.split() if word not in stops]))
    return df

def preprocess(df,cols):
    df[cols] = num_remove(df,cols)[cols]
    df[cols] = punc_remove(df,cols)[cols]
    df[cols] = lower(df,cols)[cols]
    df[cols] = space_remove(df,cols)[cols]
    df[cols] = stop_remove(df,cols)[cols]
    return df

In [53]:
df_text = preprocess(df_text, ['overview'])
df_text.head()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/13272 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/13272 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/13272 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/13272 [00:00<?, ?it/s]

,userId,movieId,rating,timestamp,imdbId,tmdbId,id,overview
0,1,296,5.0,1147880044,110912,680,680,pulp fiction burger loving hit man philosophic...
1,1,306,3.5,1147868817,111495,110,110,three colors red valentine student model genev...
2,1,307,5.0,1147868828,108394,108,108,three colors blue julie haunted grief living t...
3,1,665,5.0,1147878820,114787,11902,11902,underground group serbian socialists prepares ...
4,1,899,3.5,1147868510,45152,872,872,singin rain hollywood silent film production c...


In [54]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.1 MB/s eta 0:00:00


In [55]:
from transformers import BertTokenizer, TFBertModel

In [56]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [57]:
max_seq_length = 128
encoded_text_data = tokenizer.batch_encode_plus(df_text['overview'].tolist(), padding=True, truncation=True, max_length=max_seq_length, return_tensors="tf")

In [58]:
encoded_text_data['input_ids'].shape

TensorShape([13272, 128])

clean metadata

In [59]:
df_metadata.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId,id,genres,runtime,popularity,...,vote_average,original_language,production_companies,production_countries,spoken_languages,is_en,year,month,day,weekday
0,1,296,5.0,1147880044,110912,680,680,7762.0,154.0,77.661,...,8.490,17287.0,280.0,14370.0,21999.0,1.0,1994.0,9.0,10.0,5.0
1,1,306,3.5,1147868817,111495,110,110,17795.0,100.0,17.682,...,7.956,1253.0,134.0,2702.0,2521.0,0.0,1994.0,5.0,12.0,3.0
2,1,307,5.0,1147868828,108394,108,108,11512.0,98.0,15.606,...,7.689,1253.0,357.0,2702.0,2742.0,0.0,1993.0,8.0,1.0,6.0
3,1,665,5.0,1147878820,114787,11902,11902,20693.0,170.0,10.753,...,7.741,11.0,7.0,4047.0,4002.0,0.0,1995.0,4.0,11.0,1.0
4,1,899,3.5,1147868510,45152,872,872,25406.0,103.0,22.318,...,8.165,17287.0,637.0,14370.0,17890.0,1.0,1952.0,4.0,9.0,2.0


In [60]:
df_info = df_metadata[['userId','timestamp','rating']]

In [61]:
# frequency encoder
user_freq = df_info['userId'].value_counts().to_dict()
df_info['userId_freq_enc'] = df_info['userId'].map(user_freq)

user_freq = df_metadata['movieId'].value_counts().to_dict()
df_metadata['movieId_freq_enc'] = df_metadata['movieId'].map(user_freq)

# target encoding
user_target_mean = df_info.groupby('userId')['rating'].mean().to_dict()
df_info['userId_target_enc'] = df_info['userId'].map(user_target_mean)
df_info.drop(['rating'], axis = 1, inplace = True)

user_target_mean = df_metadata.groupby('movieId')['rating'].mean().to_dict()
df_metadata['movieId_target_enc'] = df_metadata['movieId'].map(user_target_mean)
df_metadata.drop(['movieId'], axis = 1, inplace = True)

<ipython-input-61-90d1e7e24487>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_info['userId_freq_enc'] = df_info['userId'].map(user_freq)
<ipython-input-61-90d1e7e24487>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_info['userId_target_enc'] = df_info['userId'].map(user_target_mean)
<ipython-input-61-90d1e7e24487>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [62]:
label = df_metadata['rating']
df_metadata.drop(['rating','id','userId','timestamp'], axis = 1, inplace = True)

split train and test

In [63]:
import tensorflow as tf
max_timestamp_idx = df_info.groupby('userId')['timestamp'].idxmax().tolist()
#X_text_train = df.loc[max_timestamp_idx]
#X_text_val = df.drop(max_timestamp_idx)
mask = [i not in max_timestamp_idx for i in range(df_metadata.shape[0])]
X_text_train = tf.boolean_mask(encoded_text_data['input_ids'], mask, axis=0)
X_text_val = tf.gather(encoded_text_data['input_ids'], max_timestamp_idx)
X_metadata_train = df_metadata.drop(max_timestamp_idx)
X_metadata_val = df_metadata.loc[max_timestamp_idx]
X_people_train = df_info.drop(max_timestamp_idx)
X_people_val = df_info.loc[max_timestamp_idx]
label_train = label.drop(max_timestamp_idx)
label_val = label.loc[max_timestamp_idx]

X_people_train.reset_index(inplace = True, drop = True)
X_metadata_train.reset_index(inplace = True, drop = True)
label_train.reset_index(inplace = True, drop = True)

second_timestamp_idx = X_people_train.groupby('userId')['timestamp'].idxmax().tolist()
X_people_test = X_people_train.loc[second_timestamp_idx]
X_people_train = X_people_train.drop(second_timestamp_idx)
X_metadata_test = X_metadata_train.loc[second_timestamp_idx]
X_metadata_train = X_metadata_train.drop(second_timestamp_idx)

mask = [i not in second_timestamp_idx for i in range(X_text_train.shape[0])]
X_text_test = tf.gather(X_text_train, second_timestamp_idx)
X_text_train = tf.boolean_mask(X_text_train, mask, axis=0)

label_test = label.loc[second_timestamp_idx]
label_train = label_train.drop(second_timestamp_idx)

In [64]:
X_people_train.reset_index(inplace = True, drop = True)
X_people_train

,userId,timestamp,userId_freq_enc,userId_target_enc
0,1,1147868817,70,3.814286
1,1,1147868828,70,3.814286
2,1,1147878820,70,3.814286
3,1,1147868510,70,3.814286
4,1,1147868495,70,3.814286
...,...,...,...,...
13067,100,862169009,61,3.508197
13068,100,862169335,61,3.508197
13069,100,862169186,61,3.508197
13070,100,862169065,61,3.508197


In [65]:
X_people_train.drop(['userId'], axis = 1, inplace = True)
X_people_val.drop(['userId'], axis = 1, inplace = True)
X_people_test.drop(['userId'], axis = 1, inplace = True)

In [66]:
print(X_people_train.shape, X_people_val.shape, X_people_test.shape)
print(X_metadata_train.shape, X_metadata_val.shape, X_metadata_test.shape)
print(X_text_train.shape,X_text_val.shape, X_text_test.shape)
print(label_train.shape, label_val.shape, label_test.shape)

(13072, 3) (100, 3) (100, 3)
(13072, 18) (100, 18) (100, 18)
(13072, 128) (100, 128) (100, 128)
(13072,) (100,) (100,)


In [67]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_metadata_train)
X_metadata_train = scaler.transform(X_metadata_train)
X_metadata_val = scaler.transform(X_metadata_val)
X_metadata_test = scaler.transform(X_metadata_test)

scaler = MinMaxScaler()
scaler.fit(X_people_train)
X_people_train = scaler.transform(X_people_train)
X_people_val = scaler.transform(X_people_val)
X_people_test = scaler.transform(X_people_test)

# Modeling

User id and movie data (similar as Model_v0_1.ipynb)

In [94]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from transformers import TFBertModel

In [95]:
# user info and movie info
user_info_input = Input(shape=(X_people_train.shape[1],), name="user_info_input")
movie_info_input = Input(shape=(X_metadata_train.shape[1],), name="movie_info_input")

# BERT
text_input = Input(shape=(max_seq_length,), dtype=tf.int32, name="text_input")
bert_model = TFBertModel.from_pretrained("bert-base-uncased")
bert_output = bert_model(text_input)[1]
bert_output = Dropout(0.1)(bert_output)
bert_output = Dense(64, activation="relu")(bert_output)
# hidden_layer = BatchNormalization()(hidden_layer)  # Add batch normalization layer
bert_output = Dropout(0.1)(bert_output)

bert_output = Dense(4, activation="relu")(bert_output)
# hidden_layer_2 = BatchNormalization()(hidden_layer_2)  # Add batch normalization layer

combined_output = Concatenate()([user_info_input, movie_info_input, bert_output])
# combined_output = BatchNormalization()(combined_output)

output = Dense(1, activation="linear", name="output")(combined_output)
# output = Dense(1, activation="linear", name="output")(hidden_layer_2)
hybrid_model = Model(inputs=[user_info_input, movie_info_input, text_input], outputs=output)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [96]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.01)
hybrid_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])

In [97]:
hybrid_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text_input (InputLayer)        [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_3 (TFBertModel)  TFBaseModelOutputWi  109482240   ['text_input[0][0]']             
                                thPoolingAndCrossAt                                               
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                               
                                 768),                                                            
                                 pooler_output=(Non                                         

In [98]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
hybrid_model.fit([X_people_train, X_metadata_train, X_text_train], 
                 label_train, 
                 validation_data=([X_people_val, X_metadata_val, X_text_val], label_val), 
                 epochs=50, 
                 batch_size=32, 
                 callbacks=[early_stopping])

Epoch 1/50
409/409 [==============================] - 116s 168ms/step - loss: 1.0756 - mean_squared_error: 1.0756 - val_loss: 0.8756 - val_mean_squared_error: 0.8756
Epoch 2/50
409/409 [==============================] - 45s 111ms/step - loss: 0.6666 - mean_squared_error: 0.6666 - val_loss: 0.6222 - val_mean_squared_error: 0.6222
Epoch 3/50
409/409 [==============================] - 43s 104ms/step - loss: 0.5750 - mean_squared_error: 0.5750 - val_loss: 0.5267 - val_mean_squared_error: 0.5267
Epoch 4/50
409/409 [==============================] - 43s 104ms/step - loss: 0.5450 - mean_squared_error: 0.5450 - val_loss: 0.5101 - val_mean_squared_error: 0.5101
Epoch 5/50
409/409 [==============================] - 42s 102ms/step - loss: 0.5336 - mean_squared_error: 0.5336 - val_loss: 0.4849 - val_mean_squared_error: 0.4849
Epoch 6/50
409/409 [==============================] - 41s 100ms/step - loss: 0.5239 - mean_squared_error: 0.5239 - val_loss: 0.4767 - val_mean_squared_error: 0.4767
Epoch 7/5

In [100]:
label_pred = hybrid_model.predict([X_people_val, X_metadata_val, X_text_val])
rmse = sqrt(mean_squared_error(label_val, label_pred))
print("Root Mean Square Error:", rmse)

4/4 [==============================] - 0s 36ms/step
Root Mean Square Error: 0.6706550443517029


In [ ]:
del hybrid_model

## Backend Usage
* system RAM: 40.6 GB
* GPU RAM: 34.4 GB
* Disk: 26 GB

It's impossible for my computer to increase the data size. Potentially if there is more System RAM and GPU RAM, the model will be better.